In [1]:
import pandas as pd

In [2]:
import tensorflow as tf

In [3]:
import string

In [4]:
import matplotlib.pyplot as plt

In [5]:
import numpy as np

In [6]:
import nltk

In [7]:
import sklearn

In [8]:
from sklearn.feature_extraction.text import CountVectorizer

In [9]:
from sklearn.linear_model import LogisticRegression

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
num_classes = 2 # 0 to 9 digits
num_features = 200 
# Training parameters.
learning_rate = 0.01
training_steps = 1000
batch_size = 256
display_step = 50

In [12]:
data_frame_original = pd.read_csv("../../dados/imdb-reviews-pt-br.csv")
data_frame_original.head()

,id,text_en,text_pt,sentiment
0,1,Once again Mr. Costner has dragged out a movie...,"Mais uma vez, o Sr. Costner arrumou um filme p...",neg
1,2,This is an example of why the majority of acti...,Este é um exemplo do motivo pelo qual a maiori...,neg
2,3,"First of all I hate those moronic rappers, who...","Primeiro de tudo eu odeio esses raps imbecis, ...",neg
3,4,Not even the Beatles could write songs everyon...,Nem mesmo os Beatles puderam escrever músicas ...,neg
4,5,Brass pictures movies is not a fitting word fo...,Filmes de fotos de latão não é uma palavra apr...,neg


In [13]:
stopwords = nltk.corpus.stopwords.words('portuguese')

In [14]:
data_frame_original['classificacao'] = data_frame_original['sentiment'].replace(['neg', 'pos'], [0, 1])
data_frame_original

,id,text_en,text_pt,sentiment,classificacao
0,1,Once again Mr. Costner has dragged out a movie...,"Mais uma vez, o Sr. Costner arrumou um filme p...",neg,0
1,2,This is an example of why the majority of acti...,Este é um exemplo do motivo pelo qual a maiori...,neg,0
2,3,"First of all I hate those moronic rappers, who...","Primeiro de tudo eu odeio esses raps imbecis, ...",neg,0
3,4,Not even the Beatles could write songs everyon...,Nem mesmo os Beatles puderam escrever músicas ...,neg,0
4,5,Brass pictures movies is not a fitting word fo...,Filmes de fotos de latão não é uma palavra apr...,neg,0
...,...,...,...,...,...
49454,49456,"Seeing as the vote average was pretty low, and...","Como a média de votos era muito baixa, e o fat...",pos,1
49455,49457,"The plot had some wretched, unbelievable twist...",O enredo teve algumas reviravoltas infelizes e...,pos,1
49456,49458,I am amazed at how this movieand most others h...,Estou espantado com a forma como este filme e ...,pos,1
49457,49459,A Christmas Together actually came before my t...,A Christmas Together realmente veio antes do m...,pos,1


In [15]:
def limpar_texto( texto ):
    return texto.lower().translate(str.maketrans("", "", string.punctuation))

In [16]:
def remover_stop_word( texto ): 
    texto_limpo = limpar_texto( texto )
    tokens = nltk.tokenize.word_tokenize(texto_limpo, language='portuguese')
    lista_limpa = []
    for token in tokens:
        if token not in stopwords:
            lista_limpa.append( token )
    return " ".join(lista_limpa)

In [17]:
data_frame_original["text_pt_no_stops"] = data_frame_original['text_pt'].apply(remover_stop_word)
data_frame_original

,id,text_en,text_pt,sentiment,classificacao,text_pt_no_stops
0,1,Once again Mr. Costner has dragged out a movie...,"Mais uma vez, o Sr. Costner arrumou um filme p...",neg,0,vez sr costner arrumou filme tempo necessário ...
1,2,This is an example of why the majority of acti...,Este é um exemplo do motivo pelo qual a maiori...,neg,0,exemplo motivo maioria filmes ação mesmos gené...
2,3,"First of all I hate those moronic rappers, who...","Primeiro de tudo eu odeio esses raps imbecis, ...",neg,0,primeiro tudo odeio raps imbecis poderiam agir...
3,4,Not even the Beatles could write songs everyon...,Nem mesmo os Beatles puderam escrever músicas ...,neg,0,beatles puderam escrever músicas todos gostass...
4,5,Brass pictures movies is not a fitting word fo...,Filmes de fotos de latão não é uma palavra apr...,neg,0,filmes fotos latão palavra apropriada verdade ...
...,...,...,...,...,...,...
49454,49456,"Seeing as the vote average was pretty low, and...","Como a média de votos era muito baixa, e o fat...",pos,1,média votos baixa fato funcionário locadora ac...
49455,49457,"The plot had some wretched, unbelievable twist...",O enredo teve algumas reviravoltas infelizes e...,pos,1,enredo algumas reviravoltas infelizes inacredi...
49456,49458,I am amazed at how this movieand most others h...,Estou espantado com a forma como este filme e ...,pos,1,espantado forma filme maioria outros média 5 e...
49457,49459,A Christmas Together actually came before my t...,A Christmas Together realmente veio antes do m...,pos,1,christmas together realmente veio antes tempo ...


In [18]:
vetorizador = CountVectorizer(max_features=num_classes)

In [19]:
bag_of_word = vetorizador.fit_transform(data_frame_original['text_pt_no_stops'])
bag_of_word.shape

(49459, 1000)

In [20]:
vocabulario = vetorizador.get_feature_names_out()
len(vocabulario)

1000

In [56]:
bag_of_word

<49459x1000 sparse matrix of type '<class 'numpy.int64'>'
	with 2567540 stored elements in Compressed Sparse Row format>

In [22]:
one_hot = pd.DataFrame.sparse.from_spmatrix(bag_of_word, columns=vetorizador.get_feature_names_out())
one_hot

,10,12,15,20,30,50,60,70,80,90,...,água,época,óbvio,ótima,ótimo,última,último,últimos,única,único
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49454,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
49455,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
49456,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
49457,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [53]:
x_train, x_test, y_train, y_test = train_test_split(bag_of_word, data_frame_original['classificacao'], random_state=50)
x_train

<37094x1000 sparse matrix of type '<class 'numpy.int64'>'
	with 1922699 stored elements in Compressed Sparse Row format>

In [54]:
train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_data = train_data.repeat().shuffle(5000).batch(batch_size).prefetch(1)

ValueError: TypeError: sparse matrix length is ambiguous; use getnnz() or shape[0]
Traceback (most recent call last):

  File "C:\Users\anton\AppData\Roaming\Python\Python310\site-packages\tensorflow\python\data\util\structure.py", line 105, in normalize_element
    spec = type_spec_from_value(t, use_fallback=False)

  File "C:\Users\anton\AppData\Roaming\Python\Python310\site-packages\tensorflow\python\data\util\structure.py", line 514, in type_spec_from_value
    raise TypeError("Could not build a `TypeSpec` for {} with type {}".format(

TypeError: Could not build a `TypeSpec` for   (0, 961)	1
  (0, 700)	1
  (0, 859)	1
  (0, 35)	1
  (0, 570)	1
  (0, 171)	1
  (0, 744)	1
  (0, 579)	1
  (0, 932)	2
  (0, 769)	1
  (0, 911)	1
  (0, 673)	1
  (0, 596)	1
  (0, 71)	1
  (0, 649)	1
  (0, 936)	1
  (0, 535)	1
  (0, 278)	1
  (0, 732)	1
  (0, 473)	1
  (0, 80)	1
  (0, 232)	1
  (0, 273)	1
  (0, 217)	1
  (0, 760)	1
  :	:
  (37092, 504)	1
  (37092, 296)	1
  (37092, 161)	1
  (37092, 348)	1
  (37092, 437)

In [39]:
# modelo
W = tf.Variable(tf.ones([num_features, num_classes]), name="weight")
b = tf.Variable(tf.zeros([num_classes]), name="bias")

In [40]:
def logistic_regression(x):
    # Apply softmax to normalize the logits to a probability distribution.
    return tf.nn.softmax(tf.matmul(x, W) + b)

In [41]:
def cross_entropy(y_pred, y_true):
    # Encode label to a one hot vector.
    y_true = tf.one_hot(y_true, depth=num_classes)
    # Clip prediction values to avoid log(0) error.
    y_pred = tf.clip_by_value(y_pred, 1e-9, 1.)
    # Compute cross-entropy.
    return tf.reduce_mean(-tf.reduce_sum(y_true * tf.math.log(y_pred)))

In [42]:
def accuracy(y_pred, y_true):
    # Predicted class is the index of the highest score in prediction vector (i.e. argmax).
    correct_prediction = tf.equal(tf.argmax(y_pred, 1), tf.cast(y_true, tf.float64))
    return tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [43]:
# Stochastic gradient descent optimizer.
optimizer = tf.optimizers.SGD(learning_rate)

In [44]:
def run_optimization(x, y):
    # Wrap computation inside a GradientTape for automatic differentiation.
    with tf.GradientTape() as g:
        pred = logistic_regression(x)
        loss = cross_entropy(pred, y)
    # Compute gradients.
    gradients = g.gradient(loss, [W, b])

    # Update W and b following gradients.
    optimizer.apply_gradients(zip(gradients, [W, b]))

In [45]:
for step, (batch_x, batch_y) in enumerate(train_data.take(training_steps), 1):
    # Run the optimization to update W and b values.
    run_optimization(batch_x, batch_y)  

    if step % display_step == 0:
        pred = logistic_regression(batch_x)
        loss = cross_entropy(pred, batch_y)
        acc = accuracy(pred, batch_y)
        print("step: %i, loss: %f, accuracy: %f" % (step, loss, acc))

InvalidArgumentError: cannot compute MatMul as input #1(zero-based) was expected to be a double tensor but is a float tensor [Op:MatMul] name: 

In [ ]:
# Visualizing the loss function
plt.plot(losses)
plt.xlabel('Num of epochs')
plt.ylabel('Loss')
plt.show()